<a href="https://colab.research.google.com/github/BumjuneKim/hymot-python/blob/main/final/final_selftest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://data.seoul.go.kr/dataList/OA-13252/F/1/datasetView.do# 사이트에서 "2021년 12월기준 공공자전거 대여소 정보", "2024년 12월기준 공공자전거 대여소 정보" xlsx파일을 다운로드 받아 데이터를 분석합니다.

# 두 파일을 비교하여 어떠한 변화가 있는지 아래와 같은 부분을 확인하여 살펴보고 그 결과를 output으로 출력해서 제출합니다.

# [참고]
# 다운로드 파일의 분석 용이성을 위해 다음과 같이 가공합니다.
# A. 1~5번째 행을 삭제하고 컬럼명을 대체할 새로운 행을 첫번째 행에 추가합니다.
# B. 첫번째 행의 컬럼명은 다음과 같이 정의합니다. (대여소번호, 대여소명, 자치구, 상세주소, 위도, 경도, 설치시기, LCD, QR, 운영방식)
# C. 대여소번호는 총 5자리의 코드로 가공합니다. 만약 원천데이터의 값이 305라면 00305로 가공합니다.


# [문제]
# 1. 2021년 12월에는 없고 2024년 12월에 새로 생긴 대여소와, 2021년에는 있었으나 2024년에는 사라진 대여소의 목록 및 개수를 구하시오.
# 2. 2021년과 2024년 기준 자치구별 공공자전거 대여소 개수를 비교하고, 가장 많이 증가한 자치구와 감소한 자치구를 각각 찾으시오
# 3. 2021년과 2024년 모두 존재하는(공통) 대여소를 기준으로, 각 대여소의 거치대수(합산, LCD+QR)가 어떻게 변했는지 계산하시오.
# 4. 2024년에 새로 생긴 대여소와 2021년 이후 사라진 대여소의 자치구별 분포를 각각 구하시오. 그리고 자치구별 신규 대여소 수와 폐쇄 대여소 수를 막대그래프 등으로 시각화하시오.

# [AI사용 부분]
# 1. 코드엔 없지만 대여소번호 가공시 총 5자리의 코드로 만들기 위해 cursor IDE에 가공을 의뢰했습니다.
# 2. 4번 문제의 matplotlib 시각화 부분을 AI로 처리하였습니다\



import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
file_2021 = '202112.xlsx'
file_2024 = '202412.xlsx'
df_2021 = pd.read_excel(file_2021)
df_2024 = pd.read_excel(file_2024)

# 컬럼명 정리 (공백 제거)
df_2021.columns = df_2021.columns.str.strip()
df_2024.columns = df_2024.columns.str.strip()

# 주요 컬럼명 지정
id_col = '대여소번호'
name_col = '대여소명'
gu_col = '자치구'
lcd_col = 'LCD'
qr_col = 'QR'

# 1. 신규/폐쇄 대여소 목록 및 개수 (대여소번호 기준)
set_2021 = set(df_2021[id_col])
set_2024 = set(df_2024[id_col])

new_stations = set_2024 - set_2021
closed_stations = set_2021 - set_2024

print(f"[1] 2024년 신규 대여소 개수: {len(new_stations)}")
print(df_2024[df_2024[id_col].isin(new_stations)][[id_col, name_col, gu_col]])

print(f"\n[1] 2021년 이후 폐쇄 대여소 개수: {len(closed_stations)}")
print(df_2021[df_2021[id_col].isin(closed_stations)][[id_col, name_col, gu_col]])

# 2. 자치구별 대여소 개수 비교 및 증감
gu_2021 = df_2021.groupby(gu_col)[id_col].nunique()
gu_2024 = df_2024.groupby(gu_col)[id_col].nunique()
gu_diff = gu_2024 - gu_2021

print("\n[2] 자치구별 대여소 개수 증감:")
print(gu_diff.sort_values(ascending=False))

most_increased_gu = gu_diff.idxmax()
most_decreased_gu = gu_diff.idxmin()
print(f"\n[2] 가장 많이 증가한 자치구: {most_increased_gu} ({gu_diff.max()}개 증가)")
print(f"[2] 가장 많이 감소한 자치구: {most_decreased_gu} ({gu_diff.min()}개 감소)")

# 3. 공통 대여소의 거치대수 변화 (대여소번호 기준)
common_stations = set_2021 & set_2024

df_2021_common = df_2021[df_2021[id_col].isin(common_stations)].set_index(id_col)
df_2024_common = df_2024[df_2024[id_col].isin(common_stations)].set_index(id_col)

df_2021_common['거치대수_합'] = df_2021_common[lcd_col].fillna(0) + df_2021_common[qr_col].fillna(0)
df_2024_common['거치대수_합'] = df_2024_common[lcd_col].fillna(0) + df_2024_common[qr_col].fillna(0)

rack_change = df_2024_common['거치대수_합'] - df_2021_common['거치대수_합']
rack_change_df = pd.DataFrame({
    '대여소명': df_2021_common[name_col],
    '2021_거치대수': df_2021_common['거치대수_합'],
    '2024_거치대수': df_2024_common['거치대수_합'],
    '증감': rack_change
})
print("\n[3] 공통 대여소의 거치대수 변화:")
print(rack_change_df)

# 4. 신규/폐쇄 대여소의 자치구별 분포 및 시각화
new_gu_dist = df_2024[df_2024[id_col].isin(new_stations)][gu_col].value_counts().sort_index()
closed_gu_dist = df_2021[df_2021[id_col].isin(closed_stations)][gu_col].value_counts().sort_index()

print("\n[4] 2024년 신규 대여소 자치구별 분포:")
print(new_gu_dist)
print("\n[4] 2021년 이후 폐쇄 대여소 자치구별 분포:")
print(closed_gu_dist)

# 시각화
plt.figure(figsize=(14, 7))
width = 0.35
labels = list(new_gu_dist.index.union(closed_gu_dist.index))
index = range(len(labels))

new_counts = new_gu_dist.reindex(labels, fill_value=0)
closed_counts = closed_gu_dist.reindex(labels, fill_value=0)

plt.bar(index, new_counts, width=width, label='신규 대여소')
plt.bar([i + width for i in index], closed_counts, width=width, label='폐쇄 대여소', color='orange')
plt.xticks([i + width/2 for i in index], labels, rotation=45)
plt.title('자치구별 신규/폐쇄 대여소 수 (2021~2024)')
plt.xlabel('자치구')
plt.ylabel('대여소 수')
plt.legend()
plt.tight_layout()
plt.show()